 # Opis notatnika
 Analiza przygotowana w poprzednim kroku dotyczyłą roku 2019. Teraz dodam również dane z 2020 roku i dokonam analizy porównawczej.

 Ten notatnik skupia się na odpowiednim przygotowaniu bazy danych pod cele raportowe.

 # Obszary raportowania odsetka opóźnień lotów
 1. Wyświetlanie TOP 10 (w sensie najwyższego odsetka opóźnień) lotnisk na podstawie liczby odlotów, również informacja o liczbie przylotów - widok `top_airports_by_departure`.  
 1. Wyświetlenie TOP 10 (w sensie najwyższego odsetka opóźnień) tras lotów. Przy czym istotna dla nas jest kolejność, przykładowo trasa (Warszawa, Paryż) jest inna niż (Paryż, Warszawa). Dodatkowym wymaganiem jest, aby minimalna liczba lotów odbytych na trasie wynosiła co najmniej 10000 przelotów - widok `top_reliability_roads`.  
 1. Porównanie roku 2019 vs. 2020, aby śledzić wpływ COVID na realizację lotów. Interesują mnie podejścia:  
     - miesiąc do miesiąca, przykładowo odsetek opoźnień styczeń 2019 vs. odsetek opoźnień styczeń 2020, odsetek opoźnień luty 2019 vs. odsetek opoźnień luty 2020 itd. - widok `year_to_year_comparision`,  
     - dzień do dnia, przykładowo odsetek opoźnień wtorek 2019 vs. odsetek opoźnień wtorek 2020 - widok `day_to_day_comparision`.  
 1. Dzienny, czyli jak danego dnia, globalnie wyglądał wskaźnik opóźnień lotów samolotu, tj. odsetek opóźnień 01-01-2019, odsetek opóźnień 02-01-2019 itd.

 # Podejście techniczne
 Moim celem będzie odseparowanie warstwy przygotowania danych (logika raportu) od warstwy prezentacyjnej (wizualizacja), aby odpowiednie procesy zajmowały się tylko swoimi zadaniami. 
 
 > W tym podejściu warstwa prezentacyjna (wykres/raport) nie implementuje logiki biznesowej w celu przetwarzania danych, czyli żeby przykładowo agregacja nie była wykonywana w momencie tworzenia wizualizacji.

 To podejście będzie spójne ze współczesnym sposobem projektowania aplikacji. Ma to też dodatkowy benefit - ze względu na wolumen danych na bazie, nie trzeba ich najpierw pobierać - warstwa logiczna je odpowiednio zagreguje i przekaże zdecydowanie mniejszą liczbę wierszy, co przyśpieszy działanie całości.

  Na bazie danych, gdzie umieszczone są już dane, wszystko zawarte jest na schemacie `public`. Dlatego tworzę schemat dedykowany - `reporting` w którym stworzę konkretne widoki.

 # Aktualizacja bazy danych

 Import wymaganych bibliotek

In [86]:
import psycopg2

 ## Konfiguracja połączenia

In [87]:
username = 'postgres'
password = '****'

host = 'localhost'
database = 'airlines'
port= '5432'

 Definiuję zmienną `con` oraz `cursor`

In [88]:
con = psycopg2.connect(
    host=host,
    user=username,
    password=password,
    dbname=database,
    port=port)

cursor = con.cursor()

In [89]:
con.autocommit = True

In [90]:
if not con.closed:
    print("Połączenie jest otwarte.")
else:
    print("Połączenie jest zamknięte.")

Połączenie jest otwarte.


 ## Wczytanie pliku `reporting.sql`
 Z katalogu `sql` wczytuję plik `reporting.sql`

 Rozdzielam zawartość pliku `reporting.sql` na mniejsze kwerendy używając `;`

In [91]:
commands = []

with open(r"..\sql\reporting.sql", 'r', encoding='UTF-8') as file:
    sql_text = file.read()
    sql_commands = sql_text.split(';')
    print(len(sql_commands))                            # jest tylko 8 kwarend, doszła jedna pusta pozycja na liście
    for command in sql_commands:
        if command:                                     # tu eliminuję jedną pustą pozycję na liście
            commands.append(command.strip() + ';')      # oczyszczam z pustych wierszy i dodaje średnik, choć to niepotrzebne

for index, command in enumerate(commands, start=1):
    print(f"Command {index}:")
    print(command)
    print("--------------------")

9
Command 1:
/*
Tutaj zdefiniuj schemę `reporting`
*/
DROP SCHEMA IF EXISTS reporting CASCADE;
--------------------
Command 2:
CREATE SCHEMA reporting;
--------------------
Command 3:
/*
Tutaj napisz definicję widoku reporting.flight, która:
- będzie usuwać dane o lotach anulowanych `cancelled = 0`
- będzie zawierać kolumnę `is_delayed`, zgodnie z wcześniejszą definicją tj. `is_delayed = 1 if dep_delay_new > 0 else 0` (zaimplementowana w SQL)

Wskazówka:
- SQL - analiza danych > Dzień 4 Proceduralny SQL > Wyrażenia warunkowe
- SQL - analiza danych > Przygotowanie do zjazdu 2 > Widoki
*/
CREATE OR REPLACE VIEW reporting.flight as
SELECT 
	*
	, CASE 
		WHEN dep_delay_new > 15 THEN 1 
		ELSE 0
	END AS is_delayed
FROM public.flight 
WHERE cancelled = '0';
--------------------
Command 4:
/*
Tutaj tworzę widok którego nie ma w notatniku 6 a jest wspomniany w notatniku 5 -> top_airports_by_departure
*/
CREATE OR REPLACE VIEW reporting.top_airports_by_departure AS
WITH cte_departure AS (
	SELE

 W tym miejscu wykonuję każdą z kwerend, aby zainicjować strukturę bazy danych

In [92]:
for i, command in enumerate(commands, start=1):
    cursor.execute(command)
    print(f"Kwerenda {i} została przeprocesowana")

Kwerenda 1 została przeprocesowana
Kwerenda 2 została przeprocesowana
Kwerenda 3 została przeprocesowana
Kwerenda 4 została przeprocesowana
Kwerenda 5 została przeprocesowana
Kwerenda 6 została przeprocesowana
Kwerenda 7 została przeprocesowana
Kwerenda 8 została przeprocesowana


 ### Sprawdzenie
 Kod poniżej sprawdza, czy ta część została poprawnie wykonana

In [94]:
def check_if_table_exists(table_name):
    msg = f"Sprawdzam czy istnieje tabela {table_name}"
    print(msg)

    query = f"select 1 from {table_name}"
    # jeżeli tabela nie istnieje, ten krok zwróci wyjątek
    cursor.execute(query)
    print('OK!')

In [95]:
views_to_test = [
    'reporting.flight',
    'reporting.top_reliability_roads',
    'reporting.year_to_year_comparision',
    'reporting.day_to_day_comparision',
    'reporting.day_by_day_reliability'
]

In [96]:
for view in views_to_test:
    check_if_table_exists(view)

Sprawdzam czy istnieje tabela reporting.flight


OK!
Sprawdzam czy istnieje tabela reporting.top_reliability_roads
OK!
Sprawdzam czy istnieje tabela reporting.year_to_year_comparision
OK!
Sprawdzam czy istnieje tabela reporting.day_to_day_comparision
OK!
Sprawdzam czy istnieje tabela reporting.day_by_day_reliability
OK!


In [ ]:
con.close()

 # Podsumownie
 W tym notatniku stworzyłam nowy schemat - `reporting`, którego zadaniem jest przygotowanie danych do wizualizacji. 
 Dalsza część pracy będzie polegała na wyświetleniu w wizualnie atrakcyjny sposób danych w interaktywnym raporcie stworzonym z pomocą `Dash`.